# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/). 

# Part One: Building a company list

Search for businesses with the word **WRECK** in their names.

* **Tip:** Start by scraping the first page to a dataframe, then expand to a loop that combines all of the pages. Finally combine all of the dataframes with `pd.concat`.
* **Tip:** There are a lot of ways to do this, although raw `pd.read_html` with a URL won't work! Some approaches are playwright-driven, some use [curlconverter](https://curlconverter.com/), etc etc. I recommend using requests and BeautifulSoup.
* **Tip:** You can't just do a `try`/`except`, because even if you ask for page 99999 it will always give you the last page again! Watch out that you don't get stuck in an infinite loop!

In [19]:
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()
url = "https://www.tdlr.texas.gov/tools_search/"
await page.goto(url)

await page.click("text=Company Name")
await page.fill("input[name='pht']", "WRECK")
await page.click("text=Search")




TimeoutError: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for locator("input[name='pht']")
============================================================

In [ ]:

#await page.locator("namebutton").click(timeout=5000)

#await page.fill('Company Name', 'WRECK')

In [ ]:
from bs4 import BeautifulSoup
import requests
my_url = "https://www.tdlr.texas.gov/tools_search/mccs_search_process.asp"
raw_html = requests.get(my_url).content

In [ ]:
import pandas as pd

tables = pd.read_html(await page.content())
len(tables)

### Cleanup

If you haven't already, rename the columns to be:
    
    * Customer
    * DBA Name
    * TDLR Number
    * City
    * State
    * Zip code

and remove all of the rows where the customer name is `Customer`.

## Save as `wreckers.csv`

# Part Two: Company info

> You can use whatever tool you'd like for this, but remember that form submission doesn't necessarily mean Selenium/Playwright! If you want to go the `requests` route instead, it might mean anything from adding user-agent headers to using [curlconverter](https://curlconverter.com/) to steal the whole headers/cookies/form details. And the only way to know is trial-and-error!

## Step 1: Scraping one page

Try searching from the [tools page](https://www.tdlr.texas.gov/tools_search/) for the TDLR Number `006556161C`. From the results page, scrape the:

* Business name
* Phone number
* License status
* Physical address

And save the results into a dictionary. It's best if each item has its own key, but **it's fine to pull "larger" sections of the page and split them up in pandas later on**

## Step 2: Converting to a function

Write a function called `get_tdlr_info` that is given a TDLR number and returns a dictionary with the business name, phone number, license status, and physical address. You'll mostly be able to use the same content as above.

Test with `0654479VSF`, and confirm that the information is in there. Did it not work out? Go back and edit your selectors, or be a little broader in the parts of the page you sweep up.

## Step 3: Scraping many pages

Using pandas, read in `trucks-subset.csv`.

## Scrape every single row, creating a new dataframe from the scraped data.

You probably want to refer to the classwork about using `.apply`.

Right now, the results from `.apply` will be a list of dictionaries. You can either change your function to `return pd.Series(data)` to make it become a dataframe automatically, or convert the list of dictionaries you end up with to a dataframe using `pd.DataFrame`.

* **Tip:** If you're using Playwright to navigate pages... it's going to be a bit more difficult.
* **Tip:** Remember to use `join` and not `merge` to combine your dataframes

## Save your dataframe as `data-uncleaned.csv`

# Cleaning your data

## Re-open the `data-uncleaned.csv` file

You probably want to set `pd.options.display.max_colwidth`

## Clean it up!

Make sure there are columns for

- Business name
- Phone number
- License status
- Physical address

And drop all of the other columns (The easiest way is to use `df.drop(columns=[...])`)

# Putting it together

## Open up `wreckers.csv` from Part 1

## Clean up the TDLR Number column to *just* the TDLR number

## Applying 

Use `.apply` to run your scraping script on all of your TDLR numbers. Save the results into a new dataframe.

* **Tip:** You can also just do this for the first 20 or so if you don't like waiting around.

### Use `.join` to combine it with the original dataframe

### Save to a CSV